# Spectroscopic Fitting for in-vivo Metabolomics

## AKA *Accurately inferring body chemistry using an MRI machine*

# Some problems:

## Non-ideal measurement

* Accurate spectroscopy requires averaging
* People move and breathe
* Suceptibility of tissue

## 

# Existing work

Not a lot of work seems to have been done yet.

### Magnetic Resonance Spectroscopy Quantification using Deep Learning
https://arxiv.org/abs/1806.07237

A realatively simple paper providing a bit of a baseline. Has a useful (introductory) generative signal model. A bit light on analysis.

### Quantification of Metabolites in Magnetic Resonance Spectroscopic Imaging Using Machine Learning
http://sci-hub.tw/10.1007/978-3-319-66179-7_53

Uses random forests, but 